In [2]:
SBOX = [
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
]
RCON = [0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36]


In [47]:
def sub_byte(byte):
    row=byte>>4
    col=byte & 0X0F
    index = (row << 4) + col
    return SBOX[index]



def sub_word(temp):
    out=[0]*4

    for i in range(4):
        out[i]=sub_byte(temp[i])
    return out

def rot_word(temp):
    out=[0]*4

    for i in range(4):
        out[i] = temp[(i + 1) % 4]
    return out
def add_round_key(state,round_key):

    for i in range(4):
        for j in range(4):
            state[i][j]=state[i][j] ^round_key[j][i]
    return state


def key_expanssion(key_bytes):
    
    w=[]
     
    for i in range(4):
        word=[]
        for j in range(4):
            word.append(key_bytes[4 *i+j])
        w.append(word)
    
    for i in range(4, 4*(10+1)):
        temp = [w[i-1][j] for j in range(4)]
        if i% 4==0:
            temp=rot_word(temp)
            temp=sub_word(temp)
            temp[0]^=RCON[i//4]
        new_word=[0]*4
        for j in range(4):
            new_word[j]=w[i-4][j] ^ temp[j]

        w.append(new_word)
    return w

def sub_bytes(state):
    for i in range(4):
        for j in range(4):
            state[i][j]=sub_byte(state[i][j])

    return state

def shift_rows(state):
    
    for i in range(1,4):
        row_values=[state[i][j]for j in range(4)]
        new_row=[0]*4
        for j in range(4):
            new_row[j]=row_values[(i+j) % 4]
        for j in range(4):
            state[i][j]=new_row[j]
    return state

def gmul(a,b):
    p=0
    aa=a
    bb=b
    for i in range(8):
        if(bb&1)==1:
            p^=aa
        high_bit_set=(aa & 0X80)!=0
        aa=(aa<<1) & 0XFF
        if high_bit_set:
            aa^=0X1B
        bb>>=1
    return p & 0XFF






def mix_single_column(col):
    a0,a1,a2,a3=col[0],col[1],col[2],col[3]
    b0=gmul(a0,2)^gmul(a1,3)^a2^a3
    b1=a0^gmul(a1,2)^gmul(a2,3)^a3
    b2=a0^a1^gmul(a2,2)^gmul(a3,3)
    b3=gmul(a0,3)^a1^a2^gmul(a3,2)
    return [b0 & 0xFF, b1 & 0xFF, b2 & 0xFF, b3 & 0xFF]

def mix_column(state):
        for i in range(4):
            col=[state[j][i]for j in range(4)]
            mixed=mix_single_column(col)
            for j in range(4):
                state[j][i]=mixed[j]
        return state
            

def byte_to_state(plaintext):
    state=[[0]*4 for _ in range(4)]
    for i in range(4):
        for j in range(4):
            state[i][j]=plaintext[i+4*j]
    return state

def aes_encryption(plaintext,key):
    
    state=byte_to_state(plaintext)
    expanded=key_expanssion(key)

    #initial round
    round_0_key=[expanded[i]for i in range(0,4)]

    state=add_round_key(state,round_0_key)

    for i in range(1,10):
        state=sub_bytes(state)
        state=shift_rows(state)
        state=mix_column(state)
        round_key=[expanded[4*i+j]for j in range(4)]
        state=add_round_key(state,round_key)

    state=sub_bytes(state)
    state=shift_rows(state)
    round_key=[expanded[40+i] for i in range(4)]
    state=add_round_key(state,round_key)
    return state

def state_to_byte(state):
    out=[]
    for i in  range(4):
        for j in range(4):
            out.append(state[j][i] & 0XFF)
    return bytes(out)

        
      

plaintext = b"HELLO_AES_WORLD!" 
key = b"MY_SECRET_128KEY"

cipher=aes_encryption(plaintext,key)
cipher=state_to_byte(cipher)
print("plaintext=",plaintext.hex())
print("Cipher Text =",cipher.hex())

plaintext= 48454c4c4f5f4145535f574f524c4421
Cipher Text = 1bcd2348abf86930e995063eb93f7cf3


In [ ]:
  new_row = [0]*4
print(new_row)
1bcd2348abf86930e995063eb93f7cf3


[0, 0, 0, 0]
